In [1]:
from src import notebook, datasets, build_models,caluclate_basis, custom_blocks, train, utils
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np 
import argparse
from tqdm import tqdm   
from torchsummary import summary

In [2]:
custom_args = argparse.Namespace(arch=['64', 'M', '128', 'M'], 
                                 batch_norm=False,
                                 bias=True,
                                 classifier_layers=[256],
                                 classifier_bias=True,
                                 classifier_dropout=0,
                                 avgpool=False,
                                 avgpool_size=[1,1],  # Adjusted avgpool size to (1, 1)
                                 dataset="mnist",
                                 input_height = 28,
                                 input_width = 28,
                                 greyscale=True,
                                 n_classes= 10,
                                 epochs=10, 
                                 batch_size=64,
                                 optimizer="Adam",
                                 weight_normalization=False,
                                 lr=0.001,
                                 criterion="CrossEntropyLoss",
                                 seed=42,
                                 save_checkpoints=False,
                                 save_dir = "",
                                 checkpoint_type = "epoch",
                                 )
# Load ResNet-18 model
def load_resnet18(n_classes):
    model = models.resnet18(pretrained=True)
    # Modify the final fully connected layer to have n_classes output units
    model.fc = nn.Linear(model.fc.in_features, n_classes)
    return model

# get dataloaders
train_loader, val_loader, test_loader = datasets.get_dataloaders(custom_args, logfile=None, summaryfile=None, log=False)
# build model
#model = load_resnet18(custom_args.n_classes)
#model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
model = build_models.build_model_from_args(custom_args)
# train
train_losses, train_accuracies, val_accuracies, test_accuracy = train.train(custom_args, model, train_loader, val_loader, test_loader)
#train.test_model(model, test_loader,device= torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
# log results
print(f"Final Test Accuracy: {test_accuracy}")
print(f"Final Validation Accuracy: {val_accuracies[-1]}")
print(f"Final Training Accuracy: {train_accuracies[-1]}")
print(f"Final Training Loss: {train_losses[-1]}")

Device: cuda


100%|██████████| 188/188 [00:10<00:00, 18.72it/s]


Epoch 1/10, Loss: 1.2784, Train Accuracy: 46.59%, Validation Accuracy: 69.33%


100%|██████████| 188/188 [00:08<00:00, 21.19it/s]


Epoch 2/10, Loss: 0.5295, Train Accuracy: 78.24%, Validation Accuracy: 79.11%


...

Epoch 9/10, Loss: 0.2345, Train Accuracy: 89.96%, Validation Accuracy: 89.33%


100%|██████████| 188/188 [00:07<00:00, 23.87it/s]


Epoch 10/10, Loss: 0.2349, Train Accuracy: 89.90%, Validation Accuracy: 88.97%


100%|██████████| 157/157 [00:03<00:00, 40.45it/s] 

Test Accuracy: 88.91%
Final Test Accuracy: 88.91
Final Validation Accuracy: 88.96666666666667
Final Training Accuracy: 89.89791666666666
Final Training Loss: 0.2349398085012411
